In [2]:
import pandas as pd
import warnings
import sys
import numpy as np
import scipy as sc
import random
from scipy import stats
from numpy import log10, sqrt

mydir = '/Users/kenlocey/GitHub/HACRP-HAIs/'
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

hac_mo = '10'

## Load files

In [2]:
cauti_file =  '/Users/kenlocey/GitHub/HACRP-HAIs/4_optimize_random_sampling_models/optimized_by_HAI_file_date/CAUTI/CAUTI_Data_opt_for_SIRs_2015_10.pkl'
clabsi_file = '/Users/kenlocey/GitHub/HACRP-HAIs/4_optimize_random_sampling_models/optimized_by_HAI_file_date/CLABSI/CLABSI_Data_opt_for_SIRs_2015_10.pkl'

tdf, tdf1, tdf2 = 0, 0, 0
cauti_df, clabsi_df = 0, 0
mrsa_df, cdi_df = 0, 0

opt_df = 0
cauti_df = pd.read_pickle(cauti_file)
cauti_df.rename(columns={
        'O/E': 'CAUTI O/P',
        'simulated O': 'CAUTI simulated O',
        'simulated O/E': 'CAUTI simulated O/P',
        'expected O': 'CAUTI expected O',
        'expected O/E': 'CAUTI expected O/P',
        'pi_opt': 'CAUTI pi_opt',
        'z_opt': 'CAUTI z_opt',
    }, inplace=True)
cauti_deciles = np.nanpercentile(cauti_df['CAUTI expected O/P'], np.arange(0, 100, 10))

clabsi_df = pd.read_pickle(clabsi_file)
clabsi_df.rename(columns={
        'O/E': 'CLABSI O/P',
        'simulated O': 'CLABSI simulated O',
        'simulated O/E': 'CLABSI simulated O/P',
        'expected O': 'CLABSI expected O',
        'expected O/E': 'CLABSI expected O/P',
        'pi_opt': 'CLABSI pi_opt',
        'z_opt': 'CLABSI z_opt',
    }, inplace=True)
clabsi_deciles = np.nanpercentile(clabsi_df['CLABSI expected O/P'], np.arange(0, 100, 10))

ls1 = list(cauti_df)
ls2 = list(clabsi_df)
ls = list(filter(lambda x:x in ls1, ls2))
df_yr = cauti_df.merge(clabsi_df, on=ls, how='outer')
del cauti_df, clabsi_df

print('There are', len(df_yr['Facility ID'].unique()), 'hospitals in 2015 HACRP having SIS scores for CAUTI, CLABSI, or both. This excludes many that only had scores for PSI-90 or that were given max scores for failing to submit data without a proper exception.')


There are 2620 hospitals in 2015 HACRP having SIS scores for CAUTI, CLABSI, or both. This excludes many that only had scores for PSI-90 or that were given max scores for failing to submit data without a proper exception.


In [4]:
main_df = pd.read_pickle('~/GitHub/HACRP-HAIs/data/Compiled_HCRIS-HACRP-HAI-RAND/Compiled_HCRIS-HACRP-HAI-RAND.pkl')
main_df = main_df[main_df['file_year'] == '2015']
main_df.sort_values(by='Facility ID', inplace=True)
print('Initial no. of hospitals in main_df:', len(main_df['Facility ID'].unique()))

main_df = main_df[(~main_df['Total HAC Score'].isin([float("NaN"), np.nan]))]
print('No. of hospitals in main_df having real HAC scores:', len(main_df['Facility ID'].unique()))

main_df = main_df[main_df['State'] != 'MD']
print('No. of these hospitals not in Maryland:', len(main_df['Facility ID'].unique()))

main_df = main_df[(main_df['CAUTI Predicted Cases'].isin([np.nan, float('NaN')])) | (main_df['CAUTI Predicted Cases'] < 1)]
main_df = main_df[(main_df['CLABSI Predicted Cases'].isin([np.nan, float('NaN')])) | (main_df['CLABSI Predicted Cases'] < 1)]
main_df = main_df[~main_df['AHRQ PSI-90 Score'].isin([np.nan, float('NaN')])]
print('No. of these hospitals without legit predicted cases for CAUTI and CLABSI:', len(main_df['Facility ID'].unique()))

main_df.head()

Initial no. of hospitals in main_df: 3336
No. of hospitals in main_df having real HAC scores: 3280
No. of these hospitals not in Maryland: 3241
No. of these hospitals without legit predicted cases for CAUTI and CLABSI: 660


,RPT_REC_NUM,IPPS interim payment (E_A_HOS_C1_72),IPPS payment (E_A_HOS_C1_59),PRVDR_CTRL_TYPE_CD,Facility ID,file_year,Line 19,Reconstructed IPPS payment (pre HAC penalty),Reconstructed HAC penalty,Reconstructed IPPS payment (post HAC penalty),HAC penalty imputed from E_A_HOS_C1_59,IPPS payment (from RAND),AHRQ PSI-90 Score,CAUTI Score,CLABSI Score,Domain 1 End Date,Domain 1 Score,Domain 1 Start Date,End Date,Domain 2 Score,Start Date,Facility Name,Fiscal Year,State,Total HAC Score,file_month,Payment Reduction,Total device days,CAUTI Urinary Catheter Days,CLABSI Device Days,CAUTI Observed Cases,CLABSI Observed Cases,CAUTI Predicted Cases,CLABSI Predicted Cases,CAUTI derived SIR,CLABSI derived SIR,CAUTI derived score,CLABSI derived score,Total HAC Score (derived),Payment Reduction (derived),Payment Reduction Reproduced?,HAC delta,CAUTI delta,CLABSI delta,AHRQ PSI-90 Footnote,CAUTI Footnote,CLABSI Footnote,Domain 1 Score Footnote,Domain 2 Score Footnote,SSI Footnote,SSI Score,Total HAC Footnote,CDI Footnote,CDI Score,MRSA Footnote,MRSA Score,MRSA patient days,CDI patient days,MRSA Observed Cases,CDI Observed Cases,MRSA Predicted Cases,CDI Predicted Cases,MRSA derived SIR,CDI derived SIR,MRSA derived score,CDI derived score,CDI delta,MRSA delta,Payment Reduction Footnote,CAUTI W Z Score,CDI W Z Score,CLABSI W Z Score,Domain 1 Footnote,Domain 2 Footnote,MRSA W Z Score,PSI-90 Footnote,PSI-90 W Z Score,SSI W Z Score,CAUTI derived Winsorized SIR,CAUTI derived W Z Score,CLABSI derived Winsorized SIR,CLABSI derived W Z Score,MRSA derived Winsorized SIR,MRSA derived W Z Score,HAC penalty (E_A_HOS_C1_7099),CDI derived Winsorized SIR,CDI derived W Z Score,PSI-90 End Date,PSI-90 Start Date,HAC penalty (imputed from RAND),file date,STATE,Dollar difference in payments (RAND vs E_A_HOS_C1_59),% Difference in payments (RAND vs E_A_HOS_C1_59),% Error in penalties (E_A_HOS_C1_7099 vs Imputed from RAND),Dollar difference in penalties (E_A_HOS_C1_7099 vs Imputed from RAND),% Error in penalties (E_A_HOS_C1_7099 vs Reconstructed HAC penalty),Dollar difference in penalties (E_A_HOS_C1_7099 vs Reconstructed HAC penalty),avg_Reconstructed HAC penalty,"HAC penalty, final"
3190,614801,1112676.0,1156685.0,4,010008,2015,1156685.0,1427362.0,14274.0,1413088.0,11566.85,NaN,6.0,NaN,NaN,2013-06-30 00:00:00,6.0,2011-07-01 00:00:00,2013-12-31,NaN,2012-01-01,CRENSHAW COMMUNITY HOSPITAL,2015.0,AL,6.0,10,No,0.0,0.0,0.0,0.0,0.0,0.000,0.000,NaN,NaN,NaN,NaN,6.0,No,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2015_10,AL,NaN,NaN,NaN,NaN,NaN,NaN,12028.500000,14274.0
3215,694269,198471.0,243521.0,2,010018,2015,227652.0,227652.0,2277.0,225375.0,2435.21,NaN,6.0,NaN,NaN,2013-06-30 00:00:00,6.0,2011-07-01 00:00:00,2013-12-31,NaN,2012-01-01,CALLAHAN EYE HOSPITAL,2015.0,AL,6.0,10,No,0.0,0.0,0.0,0.0,0.0,0.000,0.000,NaN,NaN,NaN,NaN,6.0,No,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2015_10,AL,NaN,NaN,NaN,NaN,NaN,NaN,3401.000000,2277.0
3234,623849,1895143.0,1885221.0,4,010022,2015,1885221.0,2262030.0,22620.0,2239410.0,18852.21,NaN,5.0,NaN,NaN,2013-06-30 00:00:00,5.0,2011-07-01 00:00:00,2013-12-31,NaN,2012-01-01,CHEROKEE MEDICAL CENTER,2015.0,AL,5.0,10,No,0.0,0.0,0.0,0.0,0.0,0.000,0.000,NaN,NaN,NaN,NaN,5.0,No,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2015_10,AL,NaN,NaN,NaN,NaN,NaN,NaN,15601.142857,22620.0
3258,632376,970627.0,959424.0,9,010032,2015,959103.0,1166812.0,11668.0,1155144.0,9594.24,NaN,5.0,NaN,NaN,2013-06-30 00:00:00,5.0,2011-07-01 00:00:00,2013-12-31,NaN,2012-01-01,WEDOWEE HOSPITAL,2015.0,AL,5.0,10,No,0.0,0.0,

In [4]:
ls1 = list(df_yr)
ls2 = list(main_df)
ls = list(filter(lambda x:x in ls1, ls2))
df_yr = df_yr.merge(main_df, on=ls, how='outer')

print(df_yr.shape)
print(len(df_yr['Facility ID'].unique()), 'hospitals resulting from merging the above dataframes.')
print('This is the same number as the number of hospitals having real HAC scores.')

(3280, 116)
3280 hospitals resulting from merging the above dataframes.
This is the same number as the number of hospitals having real HAC scores.


## Reproduce scores for 2015

In [5]:
hais = ['CAUTI', 'CLABSI']

for hai in hais:
    
    reported_scores = df_yr[hai + ' Score'].tolist()
    pred_cases = df_yr[hai + ' Observed Cases'].tolist()
    
    max_ = np.nanmax(reported_scores)
    maxS = []
    
    for i, score in enumerate(reported_scores):
        p = pred_cases[i]
        
        if p in [np.nan, float('NaN')] or np.isnan(p) == True or p < 1:
            if score == max_:
                maxS.append(1)
            else:
                maxS.append(0)
        else:
            maxS.append(0)
    
    derived_sis = df_yr[hai + ' expected O/P'].tolist()
    derived_scores = []

    for i, sis in enumerate(derived_sis):
        sis = float(sis)
        
        if maxS[i] == 1:
            #print('max score')
            derived_scores.append(10)
        
        elif hai == 'CLABSI':
            score = np.nan
            if sis <= clabsi_deciles[0]:
                score = 1
            elif sis <= clabsi_deciles[1]:
                score = 2
            elif sis <= clabsi_deciles[2]:
                score = 3
            elif sis <= clabsi_deciles[3]:
                score = 4
            elif sis <= clabsi_deciles[4]:
                score = 5
            elif sis <= clabsi_deciles[5]:
                score = 6
            elif sis <= clabsi_deciles[6]:
                score = 7
            elif sis <= clabsi_deciles[7]:
                score = 8
            elif sis <= clabsi_deciles[8]:
                score = 9
            elif sis > clabsi_deciles[8]:
                score = 10
            else:
                score = np.nan                
            derived_scores.append(score)
                
        elif hai == 'CAUTI':
            score = np.nan
            if sis <= cauti_deciles[0]:
                score = 1
            elif sis <= cauti_deciles[1]:
                score = 2
            elif sis <= cauti_deciles[2]:
                score = 3
            elif sis <= cauti_deciles[3]:
                score = 4
            elif sis <= cauti_deciles[4]:
                score = 5
            elif sis <= cauti_deciles[5]:
                score = 6
            elif sis <= cauti_deciles[6]:
                score = 7
            elif sis <= cauti_deciles[7]:
                score = 8
            elif sis <= cauti_deciles[8]:
                score = 9
            elif sis > cauti_deciles[8]:
                score = 10
            else:
                score = np.nan               
            derived_scores.append(score)
            
    df_yr[hai + ' random-based score'] = derived_scores
    print('len:', len(derived_scores), len(reported_scores))
    
features = ['CAUTI derived SIR', 'CAUTI Score', 'CAUTI derived score',
            'CLABSI derived SIR', 'CLABSI Score', 'CLABSI derived score',
            'CAUTI random-based score', 'CLABSI random-based score',
            'AHRQ PSI-90 Score']

for f in features:
    df_yr[f] = pd.to_numeric(df_yr[f], errors='coerce')

len: 3280 3280
len: 3280 3280


In [6]:
holdout_df = df_yr[(df_yr['State'] == 'MD') | (~df_yr['Payment Reduction'].isin(['Yes', 'No'])) | (df_yr['Total HAC Score'].isin([float("NaN"), np.nan]))]
df_yr = df_yr[(df_yr['State'] != 'MD') & (df_yr['Payment Reduction'].isin(['Yes', 'No'])) & (~df_yr['Total HAC Score'].isin([float("NaN"), np.nan]))]

print(holdout_df.shape[0])
print(df_yr.shape[0])
print(holdout_df.shape[0] + df_yr.shape[0])


39
3241
3280


In [7]:
hac_scores = []
ct1 = 0
ct2 = 0

for hosp in df_yr['Facility ID'].tolist():
    tdf = df_yr[df_yr['Facility ID'] == hosp]

    d2 = 0
    w_ls = []
    sum_ls = []

    # Use original scores for CAUTI and CLABSI
    #m_ls = ['CAUTI Score', 'CLABSI Score']
    
    # Use derived scores for CAUTI and CLABSI
    m_ls = ['CAUTI random-based score', 'CLABSI random-based score']
                
    s = 0
    w = 0
    for m in m_ls:
        v = tdf[m].iloc[0]

        if np.isnan(v) == False: 
            s += v
            w += 1

    if w == 0:
        d2 = np.nan
    else:
        d2 = s/w
        
    d1 = float(tdf['AHRQ PSI-90 Score'].iloc[0])
    
    if np.isnan(d1) == True: 
        # if no score for Domain 1, then total HAC score will be based entirely on Domain 2
        if np.isnan(d2) == True:
            hac_scores.append(np.nan)
        
        elif np.isnan(d2) == False:
            hac_scores.append(d2)
        
    elif np.isnan(d1) == False:
        # if there is a score for Domain 1 ...
        
        # If the derived score is NaN
        if np.isnan(d2) == True:
            hac_scores.append(d1)
        
        # If the derived score is a float
        elif np.isnan(d2) == False:
            hac_scores.append(0.35*d1 + 0.65*d2)        
    

In [8]:
df_yr['Total HAC Score (SIS-based)'] = hac_scores
print(df_yr.shape[0], 'hospitals in hac_df')
print(len(df_yr['Facility ID'].unique()), 'hospitals in hac_df')

p75 = np.nanpercentile(df_yr['Total HAC Score (SIS-based)'], 75, method='linear')
print('p75:', p75)
x = df_yr['Total HAC Score (SIS-based)'].tolist()
print(x.count(p75))

tdf = int()
pr = []
for hosp in df_yr['Facility ID'].unique():
    tdf = df_yr[df_yr['Facility ID'] == hosp]

    if tdf.shape[0] > 1: 
        print('Error:')
        print('rows:', tdf.shape[0])   
        break
        
    p = tdf['Payment Reduction'].iloc[0]
    if p != 'Yes' and p != 'No' and np.isnan(p) == True:
        pr.append(np.nan)

    else:
        score = tdf['Total HAC Score (SIS-based)'].iloc[0]

        if np.isnan(score) == True:
            pr.append('No')
        elif score <= p75:
            pr.append('No')
        elif score > p75:
            pr.append('Yes')
        else:
            print('This score is an error:', score)
            sys.exit()

            
df_yr['Payment Reduction (SIS-based)'] = pr

3241 hospitals in hac_df
3241 hospitals in hac_df
p75: 7.324999999999999
15


In [9]:
tdf1 = df_yr[df_yr['Payment Reduction (SIS-based)'] == 'Yes']
tdf2 = df_yr[df_yr['Payment Reduction (SIS-based)'].isin(['Yes', 'No'])]
print(tdf1.shape[0]/tdf2.shape[0])

tdf1 = df_yr[df_yr['Payment Reduction (SIS-based)'] == 'No']
print(tdf1.shape[0]/tdf2.shape[0])

print(tdf2.shape[0])
print(len(tdf2['Facility ID'].unique()))

0.24930576982412836
0.7506942301758717
3241
3241


In [10]:
tdf1 = df_yr[df_yr['Payment Reduction'] == 'Yes']
tdf2 = df_yr[df_yr['Payment Reduction'].isin(['Yes', 'No'])]
print(tdf1.shape[0]/tdf2.shape[0])

tdf1 = df_yr[df_yr['Payment Reduction'] == 'No']
print(tdf1.shape[0]/tdf2.shape[0])

print(df_yr.shape[0])
print(len(df_yr['Facility ID'].unique()))

0.2203023758099352
0.7796976241900648
3241
3241


In [11]:
ls1 = list(df_yr)
ls2 = list(holdout_df)
ls = list(filter(lambda x:x in ls1, ls2))
print(df_yr.shape)
print(holdout_df.shape)
df_yr = df_yr.merge(holdout_df, how='outer', on=ls)
print(df_yr.shape)

(3241, 120)
(39, 118)
(3280, 120)


In [12]:
p_np_df = df_yr[(df_yr['Payment Reduction'] == 'Yes') & (df_yr['Payment Reduction (SIS-based)'] == 'No')]
ip_ = np.round(np.nansum(p_np_df['HAC penalty, final']))
print(p_np_df.shape[0], 'hospitals were penalized but should not have been.')

np_p_df = df_yr[(df_yr['Payment Reduction'] == 'No') & (df_yr['Payment Reduction (SIS-based)'] == 'Yes')]
is_ = np.round(np.nansum(np_p_df['HAC penalty, final']))
print(np_p_df.shape[0], 'hospitals were NOT penalized but should have been.')

print(ip_, 'dollars of inappropriate penalties')
print(is_, 'dollars of inappropriate hospital savings')
print('', ip_ - is_)


79 hospitals were penalized but should not have been.
173 hospitals were NOT penalized but should have been.
44373083.0 dollars of inappropriate penalties
36963628.0 dollars of inappropriate hospital savings
 7409455.0


In [13]:
#df_yr.to_pickle('~/GitHub/HACRP-HAIs/data/finalized/final_2015.pkl', protocol=5)